In [1]:
import joblib
import pandas as pd
import keras
from sklearn.preprocessing import StandardScaler
import numpy as np

2025-10-04 11:12:08.647996: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
filename = "tess_db.csv"
df = pd.read_csv(filename, comment='#')
cols_to_drop = [
    "rowid",'tfopwg_disp', "toi", "toipfx", "tid", "ctoi_alias", "pl_pnum",
    "rastr", "raerr1", "raerr2", "decstr", "dec", "decerr1", "decerr2",
    "st_pmralim", "st_pmrasymerr",
    "st_pmdeclim", "st_pmdecsymerr",
    "pl_tranmidlim", "pl_tranmidsymerr",
    "pl_orbperlim", "pl_orbpersymerr",
    "pl_trandurhlim", "pl_trandurhsymerr",
    "pl_trandeplim", "pl_trandepsymerr",
    "pl_radelim", "pl_radesymerr",
    "pl_insolerr1", "pl_insolerr2", "pl_insollim", "pl_insolsymerr",
    "pl_eqterr1", "pl_eqterr2", "pl_eqtlim", "pl_eqtsymerr",
    "st_tmaglim", "st_tmagsymerr",
    "st_distlim", "st_distsymerr",
    "st_tefflim", "st_teffsymerr",
    "st_logglim", "st_loggsymerr",
    "st_radlim", "st_radsymerr",
    "toi_created", "rowupdate"
]
df_clean = df.drop(columns=cols_to_drop).reset_index(drop=True)

In [3]:
Y = df['tfopwg_disp'].map({'FP': 0, 'FA': 0, 'CP': 1, 'KP': 1})
X = df.drop(columns=cols_to_drop)
X_filled = X.fillna(0)
X_encoded = pd.get_dummies(X_filled, drop_first=False).astype(np.float32)

mask = Y.isna()
X_encoded = X_encoded[mask]

In [4]:
scaler = joblib.load('tess_scaler.pkl')
X_scaled = scaler.transform(X_encoded).astype(np.float32)

labels = ['FALSE POSITIVE', 'CONFIRMED']
model = keras.models.load_model('tess.keras')
pred_org = model.predict(X_scaled)
pred = (pred_org >= 0.5).astype(int).flatten()

2025-10-04 11:12:11.256995: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-10-04 11:12:11.257042: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:171] verbose logging is disabled. Rerun with verbose logging (usually --v=1 or --vmodule=cuda_diagnostics=1) to get more diagnostic output from this module
2025-10-04 11:12:11.257046: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:176] retrieving CUDA diagnostic information for host: MSI
2025-10-04 11:12:11.257048: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:183] hostname: MSI
2025-10-04 11:12:11.257171: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:190] libcuda reported version is: 580.65.6
2025-10-04 11:12:11.257208: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:194] kernel reported version is: 580.65.6
2025-10-04 11:1

161/161 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [5]:
candidates_meta = df.loc[mask, ['toi']]

with open('tess_predictions.csv', 'w') as f:
    f.write('toi,tfopwg_disp_pred,tfopwg_disp_pred_value\n')
    for i, (_, row) in enumerate(candidates_meta.iterrows()):
        f.write(f"{row['toi']},{labels[pred[i]]},{pred_org[i][0]}\n")